The Top 3 itemids (in a list) from the ‘Official Shop’ of that particular brand that generated the highest Gross Sales Revenue from 10th May to 31st May 2019.

For each brand, name the Brand and its associated top 3 items ex: (3M, 1464762206, 2218766590, 1464762339). The Brands must be sorted alphabetically in ascending order. The output file format is csv and should have 2 columns: index column and a answer column.



In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
os.listdir()

['Extra_material_2.csv',
 'output.csv',
 'Untitled.ipynb',
 'Extra_material_3.csv',
 '.ipynb_checkpoints']

In [4]:
shop = pd.read_csv("Extra_material_2.csv")
order = pd.read_csv("Extra_material_3.csv")

In [5]:
shop = shop.drop_duplicates()
order = order.drop_duplicates()

In [23]:
brand = shop.brand.drop_duplicates().sort_values(ascending=True)
brand

40                    3M
64           3M Littmann
1074                 AHC
586                 ASUS
235               Abbott
              ...       
31      Wyeth Nutrition 
59                Xiaomi
600          Yves Rocher
242                   Za
612                eMart
Name: brand, Length: 270, dtype: object

In [20]:
shop['brand_lower'] = shop.brand.str.lower()
brand = shop[['brand','brand_lower']].drop_duplicates().sort_values(by='brand_lower',ascending=True)
brand.drop(columns='brand_lower',inplace=True)
brand=brand['brand']
brand

40                    3M
64           3M Littmann
235               Abbott
57                  Acer
638               Adidas
              ...       
2180             Wrigley
31      Wyeth Nutrition 
59                Xiaomi
600          Yves Rocher
242                   Za
Name: brand, Length: 270, dtype: object

In [24]:
shop.shop_type.unique()

array(['Official Shop', 'Authorized Reseller', 'Shopee Store', nan,
       'Shopee Retail'], dtype=object)

In [25]:
shop.shop_type = shop['shop_type'].replace('Official Shop (not mall)','Official Shop')

In [26]:
merged = pd.merge(order,shop,left_on='shopid',right_on='shop_id')

In [27]:
official_order = merged[merged.shop_type.map(lambda x:x=='Official Shop')]
official_order.head()

,orderid,itemid,date_id,amount,item_price_usd,shopid,shop_id,brand,shop_type,brand_lower
0,1318797270,1488200507,16/5/2019,1,5.246674,89960894,89960894.0,P&G,Official Shop,p&g
3,1318728205,1863138742,16/5/2019,1,0.619280,89960894,89960894.0,P&G,Official Shop,p&g
6,1365475531,2052063870,26/5/2019,1,4.644596,89960894,89960894.0,P&G,Official Shop,p&g
9,1321288639,1488183688,16/5/2019,2,4.954236,89960894,89960894.0,P&G,Official Shop,p&g
12,1320036366,2052063976,16/5/2019,2,6.308910,89960894,89960894.0,P&G,Official Shop,p&g


In [28]:
official_order['date_id']=pd.to_datetime(official_order['date_id'], format="%d/%m/%Y")
start_date = pd.to_datetime('10/05/2019', format="%d/%m/%Y")
end_date = pd.to_datetime('31/05/2019', format="%d/%m/%Y")

date_check=lambda x: (x>=start_date) & (x <=end_date)
official_order=official_order[official_order.date_id.map(date_check)]
official_order

<ipython-input-28-96404e1b1816>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  official_order['date_id']=pd.to_datetime(official_order['date_id'], format="%d/%m/%Y")


,orderid,itemid,date_id,amount,item_price_usd,shopid,shop_id,brand,shop_type,brand_lower
0,1318797270,1488200507,2019-05-16,1,5.246674,89960894,89960894.0,P&G,Official Shop,p&g
3,1318728205,1863138742,2019-05-16,1,0.619280,89960894,89960894.0,P&G,Official Shop,p&g
6,1365475531,2052063870,2019-05-26,1,4.644596,89960894,89960894.0,P&G,Official Shop,p&g
9,1321288639,1488183688,2019-05-16,2,4.954236,89960894,89960894.0,P&G,Official Shop,p&g
12,1320036366,2052063976,2019-05-16,2,6.308910,89960894,89960894.0,P&G,Official Shop,p&g
...,...,...,...,...,...,...,...,...,...,...
26923,1345825831,1314524952,2019-05-22,1,81.280436,58608593,58608593.0,Honor,Official Shop,honor
26925,1347079054,959954111,2019-05-22,1,131.166842,58608593,58608593.0,Honor,Official Shop,honor
26927,1346647077,959954111,2019-05-22,1,131.166842,58608593,58608593.0,Honor,Official Shop,honor
26931,1300488429,1314524952,2019-05-11,1,79.560216,58608593,58608593.0,Honor,Official Shop,honor


In [29]:
official_order['sales']=official_order['amount']*official_order['item_price_usd']

In [30]:
official_order = official_order.groupby(["brand","itemid"]).agg({'sales':sum})['sales'].\
groupby(level=0,group_keys=False).nlargest(3)

In [31]:
official_order.head()

brand   itemid    
AHC     1617870234    117.835130
        1617870245     50.832527
        1617870178     32.125125
Abbott  1437756029    268.195336
        1414181351    238.465619
Name: sales, dtype: float64

In [32]:
", ".join(official_order.loc['AHC'].index.values.astype(str))

'1617870234, 1617870245, 1617870178'

In [33]:
result = pd.DataFrame(columns=['Index','Answers'])
ind = 1
for index,brand_name in brand.items():
    if brand_name not in official_order.index:
        answer = f'{brand_name}, N.A'
    else:
        answer = f'{brand_name}, {", ".join(official_order.loc[brand_name].index.values.astype(str))}'
#     print(answer)
    result = result.append(pd.DataFrame(data={'Index':ind,'Answers':[answer]}),ignore_index=True)
    ind+=1

In [34]:
result.to_csv("output.csv",index=False)  

In [4]:
df1.head()

,shop_id,brand,shop_type
0,92567513.0,JBL,Official Shop
1,1657263.0,Sandisk,Official Shop
2,83401047.0,Revlon,Official Shop
3,4259661.0,Philips,Official Shop
4,4350106.0,Kimberly Clark,Official Shop


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190754 entries, 0 to 190753
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   shop_id    190754 non-null  float64
 1   brand      190754 non-null  object 
 2   shop_type  124270 non-null  object 
dtypes: float64(1), object(2)
memory usage: 4.4+ MB


In [6]:
df1.shop_id = df1.shop_id.astype('int64')

In [7]:
df1_official = df1.loc[df1.shop_type=="Official Shop"]
df1_official.head()

,shop_id,brand,shop_type
0,92567513,JBL,Official Shop
1,1657263,Sandisk,Official Shop
2,83401047,Revlon,Official Shop
3,4259661,Philips,Official Shop
4,4350106,Kimberly Clark,Official Shop


In [11]:
df1_official.rename(columns={'shop_id':'shopid'})

,shopid,brand,shop_type
0,92567513,JBL,Official Shop
1,1657263,Sandisk,Official Shop
2,83401047,Revlon,Official Shop
3,4259661,Philips,Official Shop
4,4350106,Kimberly Clark,Official Shop
...,...,...,...
189578,13448324,Johnson & Johnson,Official Shop
189579,14171993,Sony,Official Shop
189580,14318452,Unilever,Official Shop
189581,14318520,Unilever,Official Shop


In [13]:
df1_official.nunique()

shopid       700
brand        246
shop_type      1
dtype: int64

In [14]:
# df1_official.groupby('brand')['shop_id'].apply(list)

In [15]:
df2 = pd.read_csv("Extra_material_3.csv")
display(df2.head())
display(df2.info())

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,16/5/2019,1,5.483204,65948755
1,1316774684,1257902741,16/5/2019,1,0.387050,61364537
2,1316803330,2126770455,16/5/2019,1,0.344044,4980129
3,1317196434,2112999088,16/5/2019,1,6.704561,29089750
4,1318105348,1832131969,16/5/2019,1,5.354187,94372493


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   orderid         1048575 non-null  int64  
 1   itemid          1048575 non-null  int64  
 2   date_id         1048575 non-null  object 
 3   amount          1048575 non-null  int64  
 4   item_price_usd  1048575 non-null  float64
 5   shopid          1048575 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 48.0+ MB


None

In [24]:
df2['date_id'] = pd.to_datetime(df2['date_id'],format="%d/%m/%Y")
df2.shape

(1048575, 6)

In [17]:
df2.head()

,orderid,itemid,date_id,amount,item_price_usd,shopid
0,1316773501,1355135789,2019-05-16,1,5.483204,65948755
1,1316774684,1257902741,2019-05-16,1,0.387050,61364537
2,1316803330,2126770455,2019-05-16,1,0.344044,4980129
3,1317196434,2112999088,2019-05-16,1,6.704561,29089750
4,1318105348,1832131969,2019-05-16,1,5.354187,94372493


In [18]:
df2_shopid = df2.loc[(df2.shopid.isin(df1_official.shopid)) & (df2.date_id>='2019-05-10') & (df2.date_id<='2019-05-31')]
df2_shopid.head()

,orderid,itemid,date_id,amount,item_price_usd,shopid
35,1318797270,1488200507,2019-05-16,1,5.246674,89960894
108,1320345215,399145157,2019-05-16,1,4.257547,16461019
178,1321836234,365980845,2019-05-16,1,7.396950,27495068
339,1320630168,1031991835,2019-05-16,1,2.021260,61792033
368,1321086927,1031991824,2019-05-16,1,2.150276,61792033


In [25]:
df2_shopid.dropna()

,orderid,itemid,date_id,amount,item_price_usd,shopid
35,1318797270,1488200507,2019-05-16,1,5.246674,89960894
108,1320345215,399145157,2019-05-16,1,4.257547,16461019
178,1321836234,365980845,2019-05-16,1,7.396950,27495068
339,1320630168,1031991835,2019-05-16,1,2.021260,61792033
368,1321086927,1031991824,2019-05-16,1,2.150276,61792033
...,...,...,...,...,...,...
976678,1377013864,2041024269,2019-05-29,1,4.515580,27495213
976784,1374335417,1710944012,2019-05-29,1,4.257547,37252407
976804,1374691551,1759004368,2019-05-29,1,25.760308,37251933
976855,1375285580,2109554227,2019-05-29,1,0.387050,111139273


In [30]:
k = pd.merge(df1_official,df2_shopid,on="shopid",how='left')
display(k.head())
k.shape

,shopid,brand,shop_type,orderid,itemid,date_id,amount,item_price_usd
0,92567513,JBL,Official Shop,NaN,NaN,NaT,NaN,NaN
1,1657263,Sandisk,Official Shop,NaN,NaN,NaT,NaN,NaN
2,83401047,Revlon,Official Shop,NaN,NaN,NaT,NaN,NaN
3,4259661,Philips,Official Shop,NaN,NaN,NaT,NaN,NaN
4,4350106,Kimberly Clark,Official Shop,NaN,NaN,NaT,NaN,NaN


(1765987, 8)

In [37]:
k['gsr'] = k.amount*k.item_price_usd

In [52]:
g = k.astype(str).groupby(['brand','itemid']).agg({'gsr':sum})

In [53]:
h = g['gsr'].groupby(level=0,group_keys=False)

In [55]:
h.head()

brand  itemid      
3M     nan             nannannannannannannannannannannannannannannann...
AHC    1617870178.0    10.7083749510.7083749510.7083749510.7083749510...
       1617870234.0    12.8586510512.8586510512.8586510512.8586510512...
       1617870245.0    9.6762424289.6762424287.2679332027.2679332027....
       1617870253.0    10.7083749510.7083749510.7083749510.7083749510...
                                             ...                        
Za     1043828695.0    8.3430712496.6658558958.3430712496.6658558958....
       1074619349.0    3.397436233.397436237.6119773766.536839333.397...
       1807609188.0    12.2995792612.2995792612.2995792612.2995792612...
       1867715485.0    11.1384301712.2995792611.1384301712.2995792611...
       1969100118.0    13.2887062713.2887062713.2887062713.2887062713...
Name: gsr, Length: 432, dtype: object

In [54]:
j = h.nlargest(3)

TypeError: Cannot use method 'nlargest' with dtype object

In [45]:
j.head()

brand   itemid      
AHC     1.617870e+09     8484.129360
        1.617870e+09     3659.941936
        1.617870e+09     2313.008989
Abbott  1.437756e+09    52834.481239
        1.414181e+09    46977.726933
Name: gsr, dtype: float64

In [51]:
for i in j.index:
    print(i)

('AHC', 1617870234.0)
('AHC', 1617870245.0)
('AHC', 1617870178.0)
('Abbott', 1437756029.0)
('Abbott', 1414181351.0)
('Abbott', 1414181383.0)
('Anessa', 2029406181.0)
('Anessa', 2029406183.0)
('Anessa', 2029406180.0)
('Anker', 2179082901.0)
('Anker', 214476390.0)
('Anker', 1985117325.0)
('Anlene', 2134752959.0)
('Anlene', 2134752955.0)
('Anlene', 2134752960.0)
('Asus', 1313801131.0)
('Asus', 1970715278.0)
('BabyLove', 1483183444.0)
('Bosch', 1285081078.0)
('Bosch', 1285081080.0)
('Bosch', 1285081090.0)
('Bosch Power Tools', 1285081078.0)
('Bosch Power Tools', 1285081080.0)
('Bosch Power Tools', 1285081090.0)
('Casio', 1768957090.0)
('Casio', 888584130.0)
('Casio', 1713698357.0)
('Coca-Cola', 2079420917.0)
('Coca-Cola', 1744510167.0)
('Coca-Cola', 1787662208.0)
('Colgate', 2278187407.0)
('Colgate', 2278187409.0)
('Colgate', 2278187434.0)
('Durex', 1426645051.0)
('Durex', 1031991832.0)
('Durex', 1310129782.0)
('Dutch Lady', 2111777546.0)
('Dutch Lady', 2111777578.0)
('Dutch Lady', 2111880

In [24]:
df2_shopid.groupby("itemid")["gsr"].sum().sort_values(ascending=False)

itemid
2047257428    61236.422731
1313801131    16079.334580
1373245117     7290.726131
2070787844     4237.721123
1720432452     3804.268467
                  ...     
2063779412        0.043006
2278187409        0.043006
2254203013        0.043006
2278187407        0.043006
2278187434        0.043006
Name: gsr, Length: 1613, dtype: float64

In [26]:
df2[df2.itemid == 2047257428]

,orderid,itemid,date_id,amount,item_price_usd,shopid,gsr
137456,1375982927,2047257428,29/5/2019,1,150.089271,65589552,150.089271
140071,1375339244,2047257428,29/5/2019,1,150.089271,65589552,150.089271
170605,1375964606,2047257428,29/5/2019,1,150.089271,65589552,150.089271
174605,1375962586,2047257428,29/5/2019,1,150.089271,65589552,150.089271
176687,1375028681,2047257428,29/5/2019,1,150.089271,65589552,150.089271
...,...,...,...,...,...,...,...
975440,1376045464,2047257428,29/5/2019,1,150.089271,65589552,150.089271
975974,1375313809,2047257428,29/5/2019,1,150.089271,65589552,150.089271
976077,1376639542,2047257428,29/5/2019,1,150.089271,65589552,150.089271
976607,1376066059,2047257428,29/5/2019,1,150.089271,65589552,150.089271


In [27]:
df1[df1.shop_id == 65589552]

,shop_id,brand,shop_type
489,65589552.0,Samsung Mobile,Official Shop
1340,65589552.0,Samsung Mobile,Official Shop
2081,65589552.0,Samsung Mobile,Official Shop
2369,65589552.0,Samsung Mobile,NaN
2579,65589552.0,Samsung Mobile,Official Shop
...,...,...,...
188877,65589552.0,Samsung Mobile,NaN
189097,65589552.0,Samsung Mobile,Official Shop
189745,65589552.0,Samsung Mobile,NaN
190051,65589552.0,Samsung Mobile,NaN
